In [3]:
!pip install ultralytics pycocotools -q

In [4]:
import os
import csv
import json
import yaml
import random
import shutil
import time
from datetime import datetime
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm as TQDM

from pycocotools.coco import COCO
import numpy as np
import pandas as pd

import torch

from ultralytics import YOLO
import ultralytics
from ultralytics.utils.files import increment_path
from ultralytics.data.converter import coco91_to_coco80_class

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
SEED = 42
EPOCHS = 50
BATCH_SIZE = 64
IMAGE_SIZE = 640
PROJECT_NAME = "yolo8n_pt_640_coco_full"
BASE_MODEL = "yolov8n.pt"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PATIENCE = 20

INPUT_DATASET_ROOT = "/kaggle/input/coco-sama-train/coco_sama"
DATASET_ROOT = "/kaggle/working/dataset"
TRAINING_ROOT = "/kaggle/working/training"

PATHS = {
    'input_root': INPUT_DATASET_ROOT,
    'dataset_root': DATASET_ROOT,
    'training_root': TRAINING_ROOT,
    'train_images': os.path.join(INPUT_DATASET_ROOT, "train"),
    'val_images': os.path.join(INPUT_DATASET_ROOT, "val"),
    'train_annotations': os.path.join(INPUT_DATASET_ROOT, "annotations", "instances_train.json"),
    'val_annotations': os.path.join(INPUT_DATASET_ROOT, "annotations", "instances_val.json"),
    'yaml_file': os.path.join(DATASET_ROOT, "yolo_main.yaml"),
    'trained_model': os.path.join(TRAINING_ROOT, PROJECT_NAME, "weights", "best.pt"),
}

print("The configuration is set:")
print(f"  - EPOCHS: {EPOCHS}")
print(f"  - BATCH_SIZE: {BATCH_SIZE}")
print(f"  - IMAGE_SIZE: {IMAGE_SIZE}")
print(f"  - DEVICE: {DEVICE}")
print(f"  - BASE_MODEL: {BASE_MODEL}")

The configuration is set:
  - EPOCHS: 50
  - BATCH_SIZE: 128
  - IMAGE_SIZE: 512
  - DEVICE: cuda
  - BASE_MODEL: yolov8n.pt


In [6]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

ultralytics.checks()

Ultralytics 8.3.167 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.3/8062.4 GB disk)


In [7]:
def custom_convert_coco(
    labels_dir: str = "../coco/annotations/",
    save_dir: str = "coco_converted/",
    cls91to80: bool = True,
    bbox_area_threshold_min: float = 0.0001,
    bbox_area_threshold_max: float = 0.8,
    use_symlinks: bool = False,
    skip_crowd_images: bool = False,
):
    """
    Convert COCO-style JSON annotations into YOLO format, filtering by bbox area
    and optionally dropping images marked with iscrowd==1.

    Parameters
    ----------
    labels_dir : str
        Path to COCO 'annotations' folder.
    save_dir : str
        Output root for 'images' and 'labels' subfolders.
    cls91to80 : bool
        Map COCO91 IDs to COCO80 if True.
    bbox_area_threshold_min : float
        Min relative bbox area (w*h / img_area) to keep an annotation.
    bbox_area_threshold_max : float
        Max relative bbox area to keep an annotation; if exceeded, drop image.
    use_symlinks : bool
        Create symlinks to the original images instead of copying.
    skip_crowd_images : bool
        If True, drop any image that has at least one annotation with iscrowd==1.
    """
    save_dir = Path(save_dir).expanduser().resolve()
    (save_dir / "labels").mkdir(parents=True, exist_ok=True)
    (save_dir / "images").mkdir(parents=True, exist_ok=True)

    coco80 = coco91_to_coco80_class()
    all_categories = {}

    n_images_saved = 0
    n_images_skipped_big_bbox = 0
    n_images_skipped_crowd = 0
    n_annotations_removed_small = 0
    n_annotations_kept = 0
    kept_train, kept_val = [], []

    for json_file in sorted(Path(labels_dir).glob("*.json")):
        split = "train" if "train" in json_file.stem.lower() else "val"
        data = json.loads(json_file.read_text(encoding="utf-8"))

        # build category lookup
        for cat in data.get("categories", []):
            all_categories[cat["id"]] = cat["name"]

        # index images and annotations by image_id
        images = {str(img['id']): img for img in data['images']}
        anns_by_image = defaultdict(list)
        crowd_by_image = defaultdict(bool)
        for ann in data['annotations']:
            img_id = str(ann['image_id'])
            anns_by_image[img_id].append(ann)
            if ann.get('iscrowd', 0) == 1:
                crowd_by_image[img_id] = True

        # process each image
        for img_id, img in TQDM(images.items(), desc=f"Processing {json_file.name}"):
            # skip if crowd flag and we want to drop crowd images
            if skip_crowd_images and crowd_by_image.get(img_id, False):
                n_images_skipped_crowd += 1
                continue

            w, h = img['width'], img['height']
            img_area = w * h
            fname = img['file_name']
            anns = anns_by_image.get(img_id, [])

            valid_anns = []
            skip_image = False

            # filter annotations by relative bbox area
            for ann in anns:
                bw, bh = ann['bbox'][2], ann['bbox'][3]
                rel = (bw * bh) / img_area
                if rel < bbox_area_threshold_min:
                    n_annotations_removed_small += 1
                    continue
                if rel > bbox_area_threshold_max:
                    skip_image = True
                    break
                valid_anns.append(ann)

            if skip_image:
                n_images_skipped_big_bbox += 1
                continue

            # record kept filename
            (kept_train if split == "train" else kept_val).append(fname)

            # prepare output directories
            img_dir = save_dir / "images" / split
            lbl_dir = save_dir / "labels" / split
            img_dir.mkdir(parents=True, exist_ok=True)
            lbl_dir.mkdir(parents=True, exist_ok=True)

            # copy or symlink the image
            src = Path(labels_dir).parent / split / fname
            dst = img_dir / fname
            if use_symlinks:
                if not dst.exists():
                    os.symlink(src, dst)
            else:
                if src.exists():
                    shutil.copy(src, dst)
                else:
                    print(f"Image not found: {src}, skipping.")

            # write YOLO-format label file
            out_f = lbl_dir / Path(fname).with_suffix(".txt")
            with open(out_f, "w", encoding="utf-8") as out:
                for ann in valid_anns:
                    box = np.array(ann['bbox'], dtype=float)
                    # convert from [x, y, w, h] to [x_center, y_center, w, h] normalized
                    box[:2] += box[2:] / 2
                    box[[0, 2]] /= w
                    box[[1, 3]] /= h
                    cls = (
                        coco80[ann['category_id'] - 1]
                        if cls91to80
                        else ann['category_id'] - 1
                    )
                    line = [cls] + box.tolist()
                    out.write(("%g " * len(line)).rstrip() % tuple(line) + "\n")

            n_images_saved += 1
            n_annotations_kept += len(valid_anns)

    # report stats
    print(f"Saved images: {n_images_saved}")
    print(f"Skipped (large bbox): {n_images_skipped_big_bbox}")
    print(f"Skipped (crowd images): {n_images_skipped_crowd}")
    print(f"Removed small anns: {n_annotations_removed_small}")
    print(f"Kept anns: {n_annotations_kept}")
    print(f"Train count: {len(kept_train)}, Val count: {len(kept_val)}")

    yaml_cats = {i - 1: n for i, n in all_categories.items()}
    return (
        save_dir,
        kept_train,
        kept_val,
        yaml_cats,
        n_images_saved,
        n_images_skipped_big_bbox,
        n_images_skipped_crowd,
        n_annotations_removed_small,
        n_annotations_kept,
    )


Annotations /kaggle/input/coco-train/coco/annotations/instances_train.json: 100%|██████████| 19500/19500 [00:02<00:00, 6872.11it/s]
Annotations /kaggle/input/coco-train/coco/annotations/instances_val.json: 100%|██████████| 5789/5789 [00:00<00:00, 6883.61it/s]

COCO data converted successfully.
Results saved to /kaggle/working/dataset


In [ ]:
save_dir, kept_tr, kept_val, yaml_cats, *rest = custom_convert_coco(
    labels_dir=os.path.join(PATHS['input_root'], "annotations"),
    save_dir=PATHS['dataset_root'],
    bbox_area_threshold_min=0.0005,
    bbox_area_threshold_max=0.7,
    use_symlinks=True,
    skip_crowd_images=True
)

# Тепер:
# - `kept_tr` та `skip_tr` — списки файлів train
# - `kept_val` та `skip_val` — списки файлів val
# - `all_cats` — {оригінальний id: назва}
# - `yaml_cats` — {id-1: назва} для запису в YAML

In [ ]:
# Creating a YAML configuration for YOLO
data_yaml = {
    "path": PATHS['dataset_root'],
    "train": "images/train",
    "val": "images/val",
    "names": yaml_cats
}

with open(PATHS['yaml_file'], "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False, allow_unicode=True)

print(f"YAML configuration is saved: {PATHS['yaml_file']}")
print("YAML configuration:")
print(yaml.dump(data_yaml, default_flow_style=False, allow_unicode=True))

In [ ]:
print(f"Model initialization: {BASE_MODEL}")
model = YOLO(BASE_MODEL)
print(f"Model initialized. Device in use: {DEVICE}")

In [ ]:
print(f"Training configuration:")
print(f"  - Epochs: {EPOCHS}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Image size: {IMAGE_SIZE}")
print(f"  - Device: {DEVICE}")
print(f"  - Patience: {PATIENCE}")

training_results = model.train(
    data=PATHS['yaml_file'],
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMAGE_SIZE,
    project=PATHS['training_root'],
    name=PROJECT_NAME,
    device=DEVICE,
    seed=SEED,
    patience=PATIENCE,
)

trained_model_path = PATHS['trained_model']
print(f"Model saved: {trained_model_path}")

In [ ]:
def create_training_readme():
    """Create a comprehensive README for the training results"""

    # Get training results path
    results_path = os.path.join(PATHS['training_root'], PROJECT_NAME)
    readme_path = os.path.join(results_path, 'README.md')

    # Read results if available
    results_csv = os.path.join(results_path, 'results.csv')

    readme_content = f"""# YOLO Training Results - {PROJECT_NAME}
## Training Configuration
**Model:** {BASE_MODEL}
**Project:** {PROJECT_NAME}
**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
### Hyperparameters
- **Epochs:** {EPOCHS}
- **Batch Size:** {BATCH_SIZE}
- **Image Size:** {IMAGE_SIZE}
- **Device:** {DEVICE}
- **Patience:** {PATIENCE}
- **Seed:** {SEED}
### Dataset Information
- **Dataset Source:** `{INPUT_DATASET_ROOT}`
- **Training Images:** {len(kept_tr)}
- **Validation Images:** {len(kept_val)}
- **Total Images:** {len(kept_tr) + len(kept_val)}
### Class Configuration
**Class Names:**
```yaml
{yaml.dump(yaml_cats, default_flow_style=False, allow_unicode=True)}```
## Training Results
"""

    # Add training metrics if results.csv exists
    if os.path.exists(results_csv):
        try:
            df = pd.read_csv(results_csv)
            last_epoch = df.iloc[-1]

            readme_content += f"""### Final Metrics (Epoch {int(last_epoch['epoch'])})
- **Train Box Loss:** {last_epoch.get('train/box_loss', 'N/A'):.4f}
- **Train Class Loss:** {last_epoch.get('train/cls_loss', 'N/A'):.4f}
- **Train DFL Loss:** {last_epoch.get('train/dfl_loss', 'N/A'):.4f}
- **Validation mAP50:** {last_epoch.get('metrics/mAP50(B)', 'N/A'):.4f}
- **Validation mAP50-95:** {last_epoch.get('metrics/mAP50-95(B)', 'N/A'):.4f}
- **Validation Precision:** {last_epoch.get('metrics/precision(B)', 'N/A'):.4f}
- **Validation Recall:** {last_epoch.get('metrics/recall(B)', 'N/A'):.4f}
### Training Progress
"""

        except Exception as e:
            readme_content += f"*Error loading training results: {str(e)}*\n\n"

    # Add file structure and environment info
    readme_content += f"""## Project Structure

## Training Environment
- **Python Version:** 3.11.11
- **PyTorch Version:** {torch.__version__}
- **Ultralytics Version:** {ultralytics.__version__}
- **CUDA Available:** {torch.cuda.is_available()}
---
*Generated automatically on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
"""

    # Write README
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme_content)

    print(f"✅ README created: {readme_path}")

    # Also create a summary JSON
    summary_data = {
        "project_name": PROJECT_NAME,
        "model": BASE_MODEL,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "image_size": IMAGE_SIZE,
        "device": DEVICE,
        "train_images": len(kept_tr),
        "val_images": len(kept_val),
        "timestamp": datetime.now().isoformat()
    }

    summary_path = os.path.join(results_path, 'training_summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary_data, f, indent=2)

    print(f"✅ Summary JSON created: {summary_path}")
    return readme_path


create_training_readme()

In [ ]:
from IPython.display import display, HTML
import os

os.chdir('/kaggle/working')

zip_filename = f"{PROJECT_NAME}_training.zip"
folder_to_zip = "training"

print(f"📦 Creating archive: {zip_filename}")

!zip -r -q {zip_filename} {folder_to_zip}/

zip_path = f'/kaggle/working/{zip_filename}'

if os.path.exists(zip_path):
    file_size = os.path.getsize(zip_path) / (1024*1024)  # у MB
    print(f"\n✅ Archive created successfully!")
    print(f"📁 File: {zip_filename}")
    print(f"📊 Size: {file_size:.1f} MB")
    print(f"📍 Path: {zip_path}")

    print(f"\n📋 Archive contents:")
    !zipinfo {zip_filename} | head -20

    display(HTML(f'''
    <div style="background-color: #e8f5e8; padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h3>📥 Download Ready</h3>
        <a href="{zip_filename}" download style="background-color: #4CAF50; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; font-weight: bold;">
            📥 Download {zip_filename} ({file_size:.1f} MB)
        </a>
    </div>
    '''))
else:
    print("❌ Error: Archive not created!")
    print("📁 Checking working directory contents:")
    !ls -la /kaggle/working/